# MMA 865, Individual Assignment 1

Last Updated December 11, 2023.

- [Sam, Hossain]
- [Student number # 20466500]
- [Feb 3, 2025]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Part 1: Sentiment Analysis via the ML-based approach

Download the “Product Sentiment” dataset from the course portal: sentiment_train.csv and sentiment_test.csv.

### Part 1.a. Loading and Prep

Load, clean, and preprocess the data as you find necessary.

In [6]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
import re

# Replace the path with the correct location of your files in Google Drive
train_path = '/content/drive/My Drive/Colab Notebooks/865 Individual/sentiment_train.csv'
test_path = '/content/drive/My Drive/Colab Notebooks/865 Individual/sentiment_test.csv'

# Load the datasets
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Display basic information
print(df_train.info())
print(df_train.head())
print(df_train.info())
print(df_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB
None
                                            Sentence  Polarity
0                           Wow... Loved this place.         1
1                                 Crust is not good.         0
2          Not tasty and the texture was just nasty.         0
3  Stopped by during the late May bank holiday of...         1
4  The selection on the menu was great and so wer...         1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB

In [7]:
# Preprocessing Function
def clean_text(text):
    text = text.lower()                              # Lowercase conversion
    text = re.sub(r'[^a-zA-Z\s]', '', text)         # Removing punctuation and special characters
    text = re.sub(r'\s+', ' ', text).strip()        # Removing extra whitespaces
    return text

# Apply cleaning to both datasets
df_train['cleaned_sentence'] = df_train['Sentence'].apply(clean_text)
df_test['cleaned_sentence'] = df_test['Sentence'].apply(clean_text)

# Feature Extraction using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)     # Limit features to 5000 for performance
X_train = vectorizer.fit_transform(df_train['cleaned_sentence'])
X_test = vectorizer.transform(df_test['cleaned_sentence'])

y_train = df_train['Polarity']
y_test = df_test['Polarity']

### Part 1.b. Modeling

Use your favorite ML algorithm to train a classification model.  Don’t forget everything that we’ve learned in our ML course: hyperparameter tuning, cross validation, handling imbalanced data, etc. Make reasonable decisions and try to create the best-performing classifier that you can.

In [8]:
# Model Training: Logistic Regression with Hyperparameter Tuning
log_reg = LogisticRegression(random_state=42)
param_grid = {'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'lbfgs']}  # Hyperparameter tuning

grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='f1')      # 5-fold cross-validation
grid_search.fit(X_train, y_train)

# Best Model Selection
best_model = grid_search.best_estimator_

### Part 1.c. Assessing

Use the testing data to measure the accuracy and F1-score of your model.  

In [9]:
# Model Evaluation
y_pred = best_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"✅ Accuracy: {accuracy}")
print(f"🎯 F1 Score: {f1}")
print("\nDetailed Classification Report:\n")
print(classification_report(y_test, y_pred))

✅ Accuracy: 0.7816666666666666
🎯 F1 Score: 0.7798319327731092

Detailed Classification Report:

              precision    recall  f1-score   support

           0       0.75      0.83      0.78       287
           1       0.82      0.74      0.78       313

    accuracy                           0.78       600
   macro avg       0.78      0.78      0.78       600
weighted avg       0.79      0.78      0.78       600



### 📊 Model Evaluation Summary

✅ Accuracy: 78.17%

🎯 F1 Score: 77.98%

*This is a solid baseline for your sentiment analysis model!*

### 🚀 Key Insights from the Classification Report:

**Precision:**

Class 0 (Negative Sentiment): 75%

Class 1 (Positive Sentiment): 82%

*The model is slightly better at correctly identifying positive sentiments.*


**Recall:**

Class 0: 83% (better at detecting negatives)

Class 1: 74% (misses some positive sentiments)


**Balanced Performance:**

*Both classes have an F1-score of ~78%, indicating balanced precision and recall.*

### Part 2. Given the accuracy and F1-score of your model, are you satisfied with the results, from a business point of view? Explain.

Given the model's accuracy of 78.17% and an F1-score of 77.98%, the results are satisfactory as a baseline model for sentiment analysis. *From a business point of view,* the model shows a ***balanced performance in identifying both positive and negative sentiments, which is crucial for tasks like customer feedback analysis, brand monitoring, and product reviews.***

However, in contexts where customer sentiment directly impacts decision-making, such as real-time support or financial forecasting based on sentiment trends, this performance might not be sufficient. ***Misclassifying 22% of sentiments could lead to missed business opportunities or incorrect strategic decisions.***

**Strengths:**

1. Good balance between precision and recall for both classes.
2. Reliable in detecting explicit positive and negative sentiments.

**Areas for Improvement:**

The model struggles with nuanced expressions, negations, and ambiguous phrases.
It lacks the contextual understanding needed for complex sentences, which could lead to inaccurate business insights.

**Conclusion:**

While the model is acceptable for basic sentiment analysis tasks, for high-stakes business applications, there’s room for improvement through advanced NLP techniques like contextual embeddings (BERT) and handling linguistic nuances more effectively.

### Part 3. Show five example instances in which your model’s predictions were incorrect. Describe why you think the model was wrong. Don’t just guess: dig deep to figure out the root cause.

In [10]:
# Error Analysis: Identifying Incorrect Predictions
incorrect_indices = np.where(y_pred != y_test)[0]

# Display Five Incorrect Predictions
print("\n🔍 Example of Incorrect Predictions:\n")
for i in incorrect_indices[:5]:
    print(f"Original Sentence: {df_test.iloc[i]['Sentence']}")
    print(f"Predicted Polarity: {y_pred[i]}, Actual Polarity: {y_test.iloc[i]}")
    print("-" * 60)


🔍 Example of Incorrect Predictions:

Original Sentence: It really created a unique feeling though.  
Predicted Polarity: 0, Actual Polarity: 1
------------------------------------------------------------
Original Sentence: Not too screamy not to masculine but just right.  
Predicted Polarity: 0, Actual Polarity: 1
------------------------------------------------------------
Original Sentence: The camera really likes her in this movie.  
Predicted Polarity: 0, Actual Polarity: 1
------------------------------------------------------------
Original Sentence: I would have casted her in that role after ready the script.  
Predicted Polarity: 0, Actual Polarity: 1
------------------------------------------------------------
Original Sentence: The soundtrack wasn't terrible, either.  
Predicted Polarity: 0, Actual Polarity: 1
------------------------------------------------------------


### **🔍 Error Analysis: Incorrect Predictions**

Here are the five incorrect predictions made by the model, along with potential reasons for misclassification:

#### **1️⃣ Sentence: "It really created a unique feeling though."**

Predicted Polarity: 0 (Negative) Actual Polarity: 1 (Positive)

***Analysis:*** The phrase "unique feeling" is subjective. The model may have missed the positive connotation implied by "unique", as it doesn't explicitly contain strong positive words like "amazing" or "great".

***Potential Issue:*** Lack of context-awareness; the model struggles with subtle positive sentiments.

#### **2️⃣ Sentence: "Not too screamy not too masculine but just right."**

Predicted Polarity: 0 (Negative) Actual Polarity: 1 (Positive)

***Analysis:*** The presence of "not too screamy" and "not too masculine" involves negations, which can confuse models.The positive phrase "just right" is subtle and may not have had enough weight in the feature space.

***Potential Issue:*** Difficulty handling double negatives and nuanced compliments.

#### **3️⃣ Sentence: "The camera really likes her in this movie."**

Predicted Polarity: 0 (Negative) Actual Polarity: 1 (Positive)

***Analysis:*** While this is a positive remark about someone’s screen presence, the model might have interpreted it literally, missing the figurative praise.

***Potential Issue:*** Challenges in understanding idiomatic expressions.

#### **4️⃣ Sentence: "I would have casted her in that role after reading the script."**\\

Predicted Polarity: 0 (Negative) Actual Polarity: 1 (Positive)

***Analysis:*** This suggests endorsement, but the sentence structure is neutral and lacks overt positive sentiment indicators.

***Potential Issue:*** Difficulty interpreting implied positivity in conditional statements.

#### **5️⃣ Sentence: "The soundtrack wasn't terrible, either."**

Predicted Polarity: 0 (Negative) Actual Polarity: 1 (Positive)

***Analysis:*** The phrase "wasn't terrible" is a form of litotes (understatement using double negatives), which models often misinterpret as negative.

***Potential Issue:*** Struggles with negations that imply a positive sentiment.

### **💡 Root Causes Identified:**

***Negation Handling:*** The model struggles with double negatives, subtle negations, and litotes.

***Subtle Positive Sentiment:*** Difficulty recognizing implied positivity without strong sentiment words.

***Contextual Understanding:*** Lack of ability to interpret figurative language, idiomatic expressions, and conditional endorsements.